# Chapter 6: Lowering to RISC-V

In [ ]:
import xdsl, riscemu

from compiler import (parse_toy, print_op, optimise_toy, lower_from_toy, 
                                  optimise_vir, lower_to_riscv, print_riscv_ssa, emulate_riscv)

example = """
def main() {
  var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
  var b<6> = [1, 2, 3, 4, 5, 6];
  var c<2, 3> = b;
  var d = a + c;
  print(d);
}
"""

toy_0 = parse_toy(example)
toy_1 = optimise_toy(toy_0)
vir_0 = lower_from_toy(toy_1)
vir_1 = optimise_vir(vir_0)
riscv = lower_to_riscv(vir_1)
code = print_riscv_ssa(riscv)

print(code)

In [ ]:
emulate_riscv(code)